# Veri Ön İşleme: Mayo LDCT Dataset

Bu notebook Mayo Clinic LDCT verisini model eğitimi için hazırlar.

**Adımlar:**
1. DICOM dosyalarını oku
2. HU (Hounsfield Unit) dönüşümü uygula
3. [-1000, 1000] aralığına clip
4. 256x256 boyutuna resize
5. [-1, 1] aralığına normalize
6. NPY formatında kaydet

---

## Neden Bu İşlemler?

CT görüntüleri ham halde piksel değerleri olarak gelir. Bunları gerçek radyolojik değerlere (HU) çevirmemiz lazım:

```
HU = pixel_value × RescaleSlope + RescaleIntercept
```

Tipik HU değerleri:
- Hava: -1000 HU
- Su: 0 HU  
- Yumuşak doku: 40-80 HU
- Kemik: 400-1000 HU

In [ ]:
import os
import pydicom
import cv2
import numpy as np

## Ayarlar

Aşağıdaki yolları kendi sisteminize göre güncelleyin.

In [ ]:
# === DOSYA YOLLARI ===
# Ham DICOM verisinin bulunduğu klasör
base_dir = r"D:\data\manifest-1755254936487\LDCT-and-Projection-data"

# İşlenmiş NPY dosyalarının kaydedileceği klasör
output_dir = r"D:\Projects\data\processed_data_npy"

os.makedirs(os.path.join(output_dir, "trainA"), exist_ok=True) # Low Dose
os.makedirs(os.path.join(output_dir, "trainB"), exist_ok=True) # High Dose

# === NORMALIZASYON AYARLARI ===
HU_MIN = -1000
HU_MAX = 1000
IMG_SIZE = (256, 256)

## Yardımcı Fonksiyonlar

In [ ]:
def find_dose_folders(patient_path):
    """
    Hasta klasöründe Low Dose ve Full Dose klasörlerini bulur.
    Projeksiyon (proj/sino) klasörlerini atlar.
    """
    low_p = None
    high_p = None

    # Tüm alt klasörleri gez
    for root, dirs, files in os.walk(patient_path):
        for d in dirs:
            d_lower = d.lower()

            # --- FİLTRE ---
            # 'proj' veya 'sino' içeren klasörleri atla
            if "proj" in d_lower or "sino" in d_lower:
                continue

            # Klasör eşleştirme
            if "low dose" in d_lower:
                low_p = os.path.join(root, d)
            elif "full dose" in d_lower or "high dose" in d_lower:
                high_p = os.path.join(root, d)

    return low_p, high_p

## Ana İşleme Döngüsü

Her hasta için:
1. Low/High dose klasörlerini bul
2. Eşleşen DICOM'ları oku
3. HU dönüşümü ve normalizasyon uygula
4. NPY olarak kaydet

In [ ]:
# C ve L ile başlayan hastaları alıyoruz
try:
    phantoms = [d for d in os.listdir(base_dir)
                if d.startswith(("C", "L"))
                and os.path.isdir(os.path.join(base_dir, d))]

    phantoms = sorted(phantoms)
    print(f"Bulunan Hastalar ({len(phantoms)} adet): {phantoms}")

except FileNotFoundError:
    print(f"HATA: '{base_dir}' yolu bulunamıyor. Lütfen yolu kontrol edin.")
    phantoms = []

print("\n--- İşlem Başlıyor ---\n")

for phantom in phantoms:
    patient_path = os.path.join(base_dir, phantom)

    # Low ve High Dose klasörlerini bul
    low_path, high_path = find_dose_folders(patient_path)

    # Klasörler bulundu mu kontrolü
    if not low_path or not high_path:
        print(f" Klasörler TAM bulunamadı (Atlanıyor -> Sadece projeksiyon olabilir): {phantom}")
        continue

    print(f" {phantom} işleniyor...")

    low_files = sorted(os.listdir(low_path))
    high_files = sorted(os.listdir(high_path))

    min_len = min(len(low_files), len(high_files))

    for i in range(min_len):
        low_f = low_files[i]
        high_f = high_files[i]

        try:
            # DICOM Oku
            low_dcm = pydicom.dcmread(os.path.join(low_path, low_f))
            high_dcm = pydicom.dcmread(os.path.join(high_path, high_f))

            # --- HU DÖNÜŞÜMÜ ---

            # Low Dose için katsayılar
            intercept_low = low_dcm.RescaleIntercept if 'RescaleIntercept' in low_dcm else 0
            slope_low = low_dcm.RescaleSlope if 'RescaleSlope' in low_dcm else 1

            # High Dose için katsayılar
            intercept_high = high_dcm.RescaleIntercept if 'RescaleIntercept' in high_dcm else 0
            slope_high = high_dcm.RescaleSlope if 'RescaleSlope' in high_dcm else 1

            # Ham veriyi HU birimine çeviriyoruz
            low_img = low_dcm.pixel_array.astype(np.float32) * slope_low + intercept_low
            high_img = high_dcm.pixel_array.astype(np.float32) * slope_high + intercept_high

            # --- NORMALIZASYON ---

            # 1. Windowing
            low_img = np.clip(low_img, HU_MIN, HU_MAX)
            high_img = np.clip(high_img, HU_MIN, HU_MAX)

            # 2. Resize
            low_img = cv2.resize(low_img, IMG_SIZE, interpolation=cv2.INTER_AREA)
            high_img = cv2.resize(high_img, IMG_SIZE, interpolation=cv2.INTER_AREA)

            # 3. Normalize (-1 ile 1 arası)
            low_img = (low_img - HU_MIN) / (HU_MAX - HU_MIN)
            high_img = (high_img - HU_MIN) / (HU_MAX - HU_MIN)

            low_img = (low_img * 2) - 1
            high_img = (high_img * 2) - 1

            # 4. Kaydet
            save_name = f"{phantom}_{i:04d}.npy"

            np.save(os.path.join(output_dir, "trainA", save_name), low_img.astype(np.float32))
            np.save(os.path.join(output_dir, "trainB", save_name), high_img.astype(np.float32))

        except Exception as e:
            print(f"   Hata ({phantom} - {i}): {e}")
            continue

    print(f"{phantom} bitti.")

print("\n TÜM İŞLEMLER BAŞARIYLA TAMAMLANDI.")